In [1]:
##

In [2]:
# import os
# # Navigate to the Open-Catalyst-Project directory
# os.chdir('/host_workspace/v9/ocp')
# # Install the package
# !pip install -e .

In [3]:
import os
import numpy as np
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import torch
import pickle
import json
import pandas as pd
from pprint import pprint
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor

In [4]:
import pickle

In [5]:
m = MPRester('Yct0KDbJbqMLWluZEovkwrLXh2VRHXbc')

/home/mchowdh5/.local/lib/python3.10/site-packages/mp_api/client/mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [8]:
try:
    config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
    checkpoint_path = "dimenetpp_all.pt"
    # Define the calculator
    calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)
except Exception as e:
    print(e)
    config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
    checkpoint_path = "dimenetpp_all.pt"
    # Define the calculator
    calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)    

In [9]:
calc

In [10]:
# with open('datasets/ocp_reactions_info.pickle', 'rb') as f:    
#     loaded_reactions = pickle.load(f)

In [11]:
# len(loaded_reactions)

In [12]:
# num_sample = len(loaded_reactions) ## len(loaded_reactions)*0 + 3 ## change here

In [13]:
## read adsorbate information

In [14]:
with open('datasets/products/d_adsid_vs_obj.pkl', 'rb') as file:
    d_adsid_vs_obj = pickle.load(file)

In [15]:
## read slab information

In [16]:
# File path for the dataset
folder_path = 'datasets/slabs/'

# File names
file_names = [
    "list_rinfo_0to100000.pkl",
    "list_rinfo_100000to200000.pkl",
    "list_rinfo_200000to300000.pkl",
    "list_rinfo_300000to400000.pkl",
    "list_rinfo_400000to500000.pkl"
]

# Initialize the combined list
list_rinfo_all = []

# Iterate through each file and append its contents to the combined list
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        list_rinfo = pickle.load(file)
        list_rinfo_all.extend(list_rinfo)

# Check the length of the combined list
len(list_rinfo_all)

443368

In [18]:
# list_rinfo_all[0].keys()

In [19]:
# d_adsid_vs_obj[0]

In [20]:
# r = loaded_reactions[0]
# pprint(r)

In [21]:
############################################################################
############################################################################
############################################################################

In [38]:
num_sample = len(list_rinfo_all)
print(num_sample)

443368


In [39]:
from collections import defaultdict
dcount = defaultdict(int)
for elem in list_rinfo_all:
    dcount[int(elem['anomaly'])] +=1

print(dcount)

defaultdict(<class 'int'>, {1: 77419, 0: 329863, 3: 15469, 4: 3851, 2: 16766})


In [40]:
aaa = AseAtomsAdaptor()
list_rinfo = []
for ir, r in enumerate(list_rinfo_all[:num_sample]):
    if ir%1000==0:
        print(ir)
    try:
        ## filter to keep only anomaly==0        
        if int(r['anomaly']) != 0:
            continue
        ###################################
        ## retrieve slab and adsorbate
        slab = r['slab']
        ase_slab = aaa.get_atoms(slab)
        ads_id = r['ads_id']
        ads_obj = d_adsid_vs_obj[ads_id]        
        ## slab + adsorbate 
        adslab = ase_slab.copy()            
        adsorbate = ads_obj
        add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
        ##################################


        ## set additional info
        tags = np.zeros(len(adslab))
        tags[18:27] = 1
        tags[27:] = 2
        adslab.set_tags(tags)
        cons= FixAtoms(indices=[atom.index for atom in adslab if (atom.tag == 0)])
        adslab.set_constraint(cons)
        adslab.center(vacuum=13.0, axis=2)
        adslab.set_pbc(True)

        ## calculator, energy, embeddings
        adslab.calc = calc
        ads_energy = adslab.get_potential_energy()
        mean_ads_embs = torch.mean(calc.trainer.model.module.embs[0], dim=0)    
        min_ads_embs = torch.min(calc.trainer.model.module.embs[0], dim=0)    
        max_ads_embs = torch.max(calc.trainer.model.module.embs[0], dim=0)    
        sum_ads_embs = torch.sum(calc.trainer.model.module.embs[0], dim=0)                

        ## additional info
        bulk_id = r['bulk_id']
        ads_id = r['ads_id']
        bulk_mpid = r['bulk_mpid']
        bulk_symbols = r['bulk_symbols']
        ads_symbols = r['ads_symbols']
        miller_index = r['miller_index']

        shift = r['shift']
        top = r['top']
        adsorption_site = r['adsorption_site']
        vclass = r['class']
        anomaly = r['anomaly']
        adslab_slab_key = r['adslab_slab_key']
        energy = r['energy']

        rinfo = {
            'ase_slab': ase_slab,            
            'adsorbate': adsorbate,
            'mean_ads_embs': mean_ads_embs,
            'min_ads_embs': min_ads_embs.values,
            'max_ads_embs': max_ads_embs.values,
            'sum_ads_embs': sum_ads_embs,

            'bulk_id': bulk_id,
            'ads_id': ads_id,
            'bulk_mpid': bulk_mpid,
            'bulk_symbols': bulk_symbols,
            'ads_symbols': ads_symbols,
            'miller_index': miller_index,

            'shift': shift,
            'top': top,
            'adsorption_site': adsorption_site,
            'class': vclass,
            'anomaly': anomaly,
            'adslab_slab_key': adslab_slab_key,
            'energy': energy,
        }
        list_rinfo.append(rinfo)
    except Exception as e:
        print(e)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [42]:
len(list_rinfo)

329863

In [43]:
with open('datasets/dict_ocp_dpp_mean_min_max_sum.pickle', 'wb') as f:
    pickle.dump(list_rinfo, f)

In [5]:
with open('datasets/dict_ocp_dpp_mean_min_max_sum.pickle', 'rb') as f:
    loaded_list_rinfo = pickle.load(f)

In [7]:
len(loaded_list_rinfo)

329863

In [11]:
loaded_list_rinfo[0]['adslab_slab_key']

'random1171399'

In [46]:
for agg_func in ['mean', 'min', 'max', 'sum']:
    print()
    print(agg_func)
    for item in loaded_list_rinfo:
        item['ads_embs'] = [float(x) for x in item[agg_func + '_ads_embs']]
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(loaded_list_rinfo)
    # Split 'ads_embs' into separate columns
    ads_embs_cols = df['ads_embs'].apply(pd.Series)
    ads_embs_cols.columns = ['e' + str(i) for i in range(256)]
    # Drop the original 'ads_embs' column and concatenate the new columns
    df = df.drop('ads_embs', axis=1)
    df = pd.concat([df, ads_embs_cols], axis=1)
    ## keep the stable reaction
    df = df.sort_values(by='energy')
    df = df.groupby(['bulk_mpid', 'miller_index', 'shift', 'ads_id']).first().reset_index()
    ##
    print(df.shape)
    print(df[['bulk_mpid', 'miller_index', 'shift', 'ads_id', 'energy']].head(2))
    df = df[['e' + str(i) for i in range(256)] + ['energy']]
    print(df.shape)
    df.to_pickle('datasets/df_ocp_dpp_' + str(agg_func) + '.pickle')


mean
(329863, 275)
  bulk_mpid miller_index  shift  ads_id    energy
0      mp-1    (1, 0, 0)   0.25       0 -0.674474
1      mp-1    (1, 0, 0)   0.25       2 -1.303317
(329863, 257)

min
(329863, 275)
  bulk_mpid miller_index  shift  ads_id    energy
0      mp-1    (1, 0, 0)   0.25       0 -0.674474
1      mp-1    (1, 0, 0)   0.25       2 -1.303317
(329863, 257)

max
(329863, 275)
  bulk_mpid miller_index  shift  ads_id    energy
0      mp-1    (1, 0, 0)   0.25       0 -0.674474
1      mp-1    (1, 0, 0)   0.25       2 -1.303317
(329863, 257)

sum
(329863, 275)
  bulk_mpid miller_index  shift  ads_id    energy
0      mp-1    (1, 0, 0)   0.25       0 -0.674474
1      mp-1    (1, 0, 0)   0.25       2 -1.303317
(329863, 257)


In [47]:
## visualize 
agg_func = 'max'
df = pd.read_pickle('datasets/df_ocp_dpp_' + str(agg_func) + '.pickle')
print(df.shape)
df.head()

(329863, 257)


,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e247,e248,e249,e250,e251,e252,e253,e254,e255,energy
0,2.361676,11.344664,11.064329,9.159813,5.177182,3.298251,8.213123,0.197204,14.385883,9.401150,...,0.595930,4.028632,5.484061,8.712776,8.624293,0.750058,6.208294,2.839834,0.985671,-0.674474
1,2.564468,11.344664,11.064329,6.575477,5.177182,10.696902,4.912156,2.199837,11.546939,9.401150,...,1.177580,4.820849,2.603472,8.712776,14.016377,10.650798,6.208294,7.757067,4.400261,-1.303317
2,3.227252,11.350498,11.064139,11.175415,5.177180,7.044148,5.048898,3.331722,7.879272,9.401955,...,1.829935,2.431113,2.605334,9.354735,9.349174,4.631165,6.209186,10.350499,6.263275,-0.382670
3,1.046619,10.866059,8.069139,9.922478,5.177182,3.298251,16.386509,-0.089941,1.264775,14.309203,...,2.003997,1.122696,10.109883,10.786431,1.953919,5.126840,2.064799,4.634651,8.414618,2.970998
4,4.313478,7.120005,8.069138,7.521814,7.533404,18.106291,7.577925,2.359809,7.291344,12.234023,...,5.597470,4.646251,5.978832,13.732367,16.198078,5.126839,4.142599,11.265613,8.607203,-0.296792


In [48]:
## visualize 
agg_func = 'mean'
df = pd.read_pickle('datasets/df_ocp_dpp_' + str(agg_func) + '.pickle')
print(df.shape)
df.head()

(329863, 257)


,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e247,e248,e249,e250,e251,e252,e253,e254,e255,energy
0,0.994186,0.814691,2.831584,0.923281,3.710807,2.404779,0.861732,-1.437050,1.650958,2.885541,...,0.303431,-1.827009,0.623609,2.870698,1.695400,0.481869,1.435603,0.401149,-1.192425,-0.674474
1,0.290946,1.059013,1.991819,0.958263,2.656359,2.276497,0.444932,-1.090392,0.415022,1.501878,...,-0.029454,-1.089820,-0.133850,2.023246,2.798918,1.206684,1.198573,1.068191,-0.578272,-1.303317
2,-0.011181,0.375714,1.867136,1.628530,1.645837,1.475136,-0.018428,-0.819613,0.705018,0.611474,...,-0.209194,-0.689756,-0.588132,1.515704,1.795644,0.490032,0.852053,2.044761,-0.370579,-0.382670
3,0.722904,1.588481,2.961112,0.835143,4.454036,2.548711,1.363413,-1.383794,-0.489730,4.895809,...,0.818593,-2.196378,1.279339,3.865371,1.296734,1.478071,1.159255,1.226252,-0.221685,2.970998
4,0.817365,0.433019,1.784142,1.327885,2.887596,3.833696,0.683231,-0.167109,0.856352,2.079584,...,0.166394,-0.592506,0.435727,3.520055,4.146413,0.456337,-0.248683,3.533665,0.255487,-0.296792
